In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)


In [2]:
#h_waste = pd.read_csv("../data/Hazardous WasteIGD_EF_RCRA.csv", low_memory=False)
toxic_release = pd.read_csv("../data/Toxic ReleasesIGD_EF_TRI.csv")
tn_pop = pd.read_csv("../data/tn_county_pop.csv")
swm_permits = pd.read_csv("../data/SWM Permits.csv")
rare_s = pd.read_csv("../data/Rare Species by County.csv", encoding='windows-1252')
#streams = pd.read_csv("../data/Exceptional TN Streams.csv", encoding='windows-1252')
streams = pd.read_excel("../data/Exceptional TN Streams.xlsx")
#rare_e = pd.read_excel("../data/Exceptional TN Streams filtered.xlsx", sheet_name='Endangered')
oil_gas = pd.read_csv("../data/Oil and Gas Well Permits.csv")


In [3]:
streams['mlat']=streams[['From Lat' , 'To Lat']].mean(axis=1)
streams['mlong']=streams[['From Long' , 'To Long']].mean(axis=1)

In [4]:
#h_waste.rename(columns={'county_name': 'county'}, inplace=True)
toxic_release.rename(columns={'latitude': 'Lat','longitude':'Long','location_address':'Address','primary_name':'Facility'}, inplace=True)
tn_pop.rename(columns={'county': 'county_name'}, inplace=True)
swm_permits.rename(columns={'County': 'county','Location':'Address'}, inplace=True)
rare_s.rename(columns={'County': 'county'}, inplace=True)
#rare_e.rename(columns={'County': 'county','From Lat':'Lat', 'From Long':'Long'}, inplace=True)
streams.rename(columns={'County': 'county','mlat':'Lat', 'mlong':'Long'}, inplace=True)
oil_gas.rename(columns={'County': 'county','Latitude':'Lat', 'Longitude':'Long','Operator Name':'Facility'}, inplace=True)


In [5]:
#h_waste['county'] = h_waste['county'].str.upper()
tn_pop['county'] = tn_pop['county_name'].str.upper()
rare_s['county'] = rare_s['county'].str.upper()
toxic_release['county'] = toxic_release['county_name'].str.upper()
streams['county'] = streams['county'].str.upper()
swm_permits['county'] = swm_permits['county'].str.upper()
oil_gas['county'] = oil_gas['county'].str.upper()


In [6]:
toxic_release['flag']="toxic_release"
tn_pop['flag']="population"
swm_permits['flag']="swm_permits"
rare_s['flag']="rare"
#rare_e['flag']="rare_endangered"
oil_gas['flag']="oil_gas"
streams['flag']="steams"


In [7]:
toxic_release = toxic_release.drop(['registry_id', 'pgm_sys_acrnm', 'pgm_sys_id','city_name','county_name','epa_region','huc_code','postal_code','facility_url','fips_code'], axis=1)
swm_permits = swm_permits.drop(['Permit Number','Permittee','City','Issuance'], axis=1)
tn_pop = tn_pop.drop([ 'stateCode','name','county_name','slug','rank','fips','state','year'], axis=1)
rare_s = rare_s.drop(['Global Rank', 'State Rank', 'Fed. Status','State Status'], axis=1)
#rare_e = rare_e.drop(['HUC', 'To Lat', 'To Long','Inclusion Date','Revision Date'], axis=1)
oil_gas = oil_gas.drop(['API No', 'Permit Date', 'Permit No','Well Name And No','Elevation','Row ID','Formation At Total Depth'], axis=1)
streams = streams.drop(['HUC','From Lat', 'To Lat','From Long' , 'To Long','Inclusion Date','Revision Date'], axis=1)



In [8]:
#tn_waste = h_waste[h_waste['state_code']=='TN']
tn_toxic = toxic_release[toxic_release['state_code']=='TN']
swm_permits1 = swm_permits[(swm_permits['Permit Type']=='CONVENIENCE')&(swm_permits['Status']=='Active')]
oil_gas1 = oil_gas[oil_gas['Purpose of Well']!='Dry & Abandoned']


In [9]:
test = pd.merge(tn_toxic, streams, on=['Lat', 'Long','county','flag'], how='outer')
test1 = pd.merge(test, oil_gas1, on=['Lat', 'Long','county','flag','Facility'], how='outer')
test2 = pd.merge(test1, swm_permits1, on=['county','flag','Facility','Address'], how='outer')
test3 = pd.merge(test2, tn_pop, on=['county','flag'], how='outer')
test4 = pd.merge(test3, rare_s, on=['county','flag'], how='outer')


In [10]:
test4

,Lat,Long,Facility,Address,state_code,county,flag,Watershed Name,Waterbody,Description,Basis for Inclusion,Purpose of Well,Permit Type,Status,pop2020,pop2024,growthSince2020,landArea,density,densityMi,areaMi,Type,Category,Scientific Name,Common Name,Habitat,Wet Habitat Flag
0,36.067778,-84.331361,"American Resource Management, Inc",NaN,NaN,ANDERSON,oil_gas,NaN,NaN,NaN,NaN,Shut-in Gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,36.055028,-84.340333,Armco Energy LLC,NaN,NaN,ANDERSON,oil_gas,NaN,NaN,NaN,NaN,Gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36.055833,-84.328583,Armco Energy LLC,NaN,NaN,ANDERSON,oil_gas,NaN,NaN,NaN,NaN,Gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,36.057139,-84.336944,Armco Energy LLC,NaN,NaN,ANDERSON,oil_gas,NaN,NaN,NaN,NaN,Gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,36.057861,-84.342444,Armco Energy LLC,NaN,NaN,ANDERSON,oil_gas,NaN,NaN,NaN,NaN,Oil And Gas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17502,36.190190,-86.273890,SOUTHTEC LLC,712 BRISKIN LANE,TN,WILSON,toxic_release,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17503,36.210278,-86.521389,UNIVERSAL RACK CO,70 ATTEND DR,TN,WILSON,toxic_release,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17504,36.123830,-86.406740,WILSON PLANT,105 DUKE DR,TN,WILSON,toxic_release,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17505,36.221990,-86.424875,WONDER PORCELAIN GROUP LLC CO LOCATED SITE NAS...,5 WONDER LANE,TN,WILSON,toxic_release,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#test4.to_csv('merged_all.csv',index=False)